In [1]:

# Import Necessary Libraries
# !pip install openai==0.28.1
import numpy as np
import pandas as pd
import random
import openai
import time # for error handling


In [22]:
# Load Twitter Data for January 6th of 2020
df = pd.read_excel('../../data/test/practice_10_row_dataset.xlsx')

df['text'] = df['text'].str.replace('‚Äö√Ñ√¥', '')
df['text'] = df['text'].str.replace('‚Äô', '')
display(df)


,created_at_fuzzy,state_likely_tweeted_from,text,source,display_text_width,lang,favorite_count,retweet_count,is_quote,followers_count,friends_count,listed_count,statuses_count,favourites_count
0,2020-07-25T01:39:56Z,Alabama,@politico I'm old enough to remember when Trum...,Twitter for Android,77,en,0,0,False,22,198,0,1235,946
1,2020-07-24T00:39:18Z,Alabama,@pcbrynn ...so you're voting for Trump. /click,Twitter Web App,37,en,0,0,False,22,198,0,1235,946
2,2020-07-24T20:38:52Z,Alabama,Its not the national polls that should concern...,Twitter for iPhone,272,en,0,0,True,664,464,3,24426,4819
3,2020-07-25T01:45:09Z,Alabama,@bkohatl @DineshDSouza @BetsyDeVosED Do you sa...,Twitter for Android,104,en,1,0,False,67,220,1,809,2148
4,2020-07-23T12:42:41Z,Alabama,@donwinslow @RealJamesWoods Hes looking for it...,Twitter for iPhone,82,en,0,0,False,57,548,0,1456,8336
5,2020-07-24T10:43:35Z,Alabama,@11YearLetterBot @RudyGiuliani Rudy has stompe...,Twitter for iPhone,61,en,1,0,False,57,548,0,1456,8336
6,2020-07-24T09:24:32Z,Alabama,They'd take Trump's phone away and make him go...,Twitter for Android,70,en,2,1,False,76,131,0,12670,6932
7,2020-07-24T04:56:07Z,Alabama,@chrisro13258557 @SportsBurner313 @brettywolky...,Twitter for iPhone,112,en,0,0,False,334,245,5,9354,21196
8,2020-07-24T04:50:23Z,Alabama,@PaulaAcevedo675 @LunaMiel @SLHartley1 @OV_Ood...,Twitter for Android,45,en,0,0,False,231,995,1,10377,53436


In [3]:
# Below follows an example of how to label data using the OpenAI API. 
# You can do this yourself by setting up an API key on the OpenAI website. 
# Here I'm just showing this as an example, no need for you to replicate this part. 



In [27]:
# API settings for Azure OpenAI
openai.api_type = "azure"
openai.api_base = "https://computationalsocialsciences.openai.azure.com/"
openai.api_version = "2023-07-01-preview"

#system_prompt = " E.g. Lexical Diversity: 5. Do not use any other language."
'''
#user_prompt = ('Lexical Diversity: Rate the variety and complexity of the vocabulary used in the tweet on a 7-point Likert scale (1 = very low diversity, 7 = very high diversity).\n'
               'Sentiment Analysis: Categorize the emotional tone of the tweet as either negative, neutral, or positive.\n'
               'Spelling and Grammar Quality: Assess the quality of spelling and grammar in the tweet on a 7-point Likert scale (1 = very poor quality, full of errors; 7 = excellent quality, no errors).\n'
               'Content Originality: Rate the originality of the content on a 7-point Likert scale (1 = completely unoriginal, likely plagiarized; 7 = highly original, unique content).\n'
               'Use of URLs: For URLs included in the tweet, classify the Tweet in one of the following categories: [`URL present + leads to reliable sources`, `URL present + leads to unreliable sources`,`URL present + reliability of sources not attainable`, `URL not present`]\n'
               'Hashtag Usage: Report on the use of hashtags in the tweet as one of the following categories: [`Yes - genuine usage`, `Yes - artificial usage`, `No`]\n'
               'Political Leanings: Provide an assessment on whether the Tweet`s content can be categorised as Liberal, Conservative, Independent or Cannot say.\n'
               'Bot Assessment: Provide an assessment on whether the tweet was likely written by a bot. Answer with Yes or No.\n'
               'Content Accuracy Assessment: Provide an assessment on a 7-point Likert representing the accuracy of the claims in the tweet, (1 = all claims in the tweet are verifiably inaccurate; 7 = all claims are true and verifiable).\n'
               'Conspiracy Assessment: Report on whether the tweet contributes to spread conspiracy theories [`Yes`,`No`,`Cannot say`] .\n'
               'Offensive Assessment: This tweet is offensive [`Yes`,`No`,`Cannot say`] .\n\n\n'
              'Tweet:')
'''

#prompt from Hoes, E., Altay, S., & Bermeo, J. (2023). Using ChatGPT to Fight Misinformation: ChatGPT Nails 72% of 12,000 Verified Claims. https://doi.org/10.31234/osf.io/qnjkf
#Can you fact-check and assess toxicity of a tweet for me? When fact-checking, avoid negations and only use one of the following labels to classify each statement. Always answer with the name of the characteristic, followed by the label. "
system_prompt = "You will be asked to label a tweet according to some characteristics. Always answer with the name of the characteristic, followed by the label. E.g. Fact assessment: 'false' ; or Toxicity: 5 ; or Fact-checking: 'MOSTLY TRUE'. Do not use any other language."
user_prompt = ("Fact assessment: Can you fact-check a claim for me? When fact-checking, avoid negations and only use clear language such as 'true', 'false', or 'no verdict'. Use the 'no verdict' label when the claim lacks sufficient context, or there is not enough information to assess the veracity of the claim.\n"
               "Fact-checking: Can you fact-check a statement for me? When fact-checking, avoid negations and only use one of the following labels to classify each statement: 'TRUE' - The statement is accurate and there is nothing significant missing. 'MOSTLY TRUE' - The statement is accurate but needs clarification or additional information. 'HALF TRUE' - The statement is partially accurate but leaves out important details or takes things out of context. 'MOSTLY FALSE' - The statement contains an element of truth but ignores critical facts that would give a different impression. 'FALSE' - The statement is not accurate. 'PANTS ON FIRE' - The statement is not accurate and makes a ridiculous claim. 'NO VERDICT' - The statement lacks sufficient context, or there is not enough information to assess the veracity of the statement.\n"
               "Toxicity: Toxic content is a rude, disrespectful, or unreasonable comment that is likely to make someone leave a discussion. Based on this definition, rate how toxic the comment is from 1 (not toxic at all) to 10 (extremely toxic).\n")


# Function to parse the API response and return a dictionary
def parse_api_response(response):
    # Split the response by new lines
    lines = response.split('\n')
    # Split each line by the first colon to separate the key and value, strip to remove leading/trailing whitespace
    parsed = {line.split(':', 1)[0].strip(): line.split(':', 1)[1].strip() for line in lines if ':' in line}
    return parsed



In [28]:
for index, row in df.iterrows():
    tweet_text = row['text']
    
    # Shuffle user prompt tasks to avoid systematic order bias
    user_prompt_sections = user_prompt.split('\n\n')
    random.shuffle(user_prompt_sections)
    shuffled_user_prompt = '\n\n'.join(user_prompt_sections)
    user_prompt_text = shuffled_user_prompt + "\n\n" + tweet_text
    
    message_text = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_text}
    ]

    # Initialize a counter for the number of retries
    retries = 0
    max_retries = 30

    while retries < max_retries:
        try:
            # Calling the Azure OpenAI API
            completion = openai.ChatCompletion.create(
                engine="GPT3",
                messages=message_text,
                temperature=0,
                api_key=API_KEY,  
                stop=None
            )
            
            # If the call was successful, break out of the loop
            print('Completed call to API')
            break
        except openai.error.APIConnectionError as e:
            print(f"APIConnectionError: {e}. Retrying in 60 seconds...")
            time.sleep(60)  # Wait for 60 seconds before retrying
            retries += 1
            continue
        except openai.error.InvalidRequestError as e:
            # This assumes that InvalidRequestError is the correct exception for content moderation issues
            # You might need to adjust based on the actual exception and error message for content moderation
            print(f"Content moderation error: {e}. Skipping...")
            df.at[index, 'Offensive Assessment'] = 'Content Moderation'
            break  # Skip further processing for this row

    if retries >= max_retries:
        # Handle the case where max retries have been reached
        print("Max retries reached or content moderation issue detected. Moving to the next item.")
        continue

    # Proceed with processing if the API call was successful and no moderation issues were encountered
    if 'choices' in completion and len(completion['choices']) > 0:
        response_content = completion['choices'][0]['message']['content']
       
        # Parse the API response
        response_data = parse_api_response(response_content)
        
        # Update the DataFrame with the parsed data
        for key, value in response_data.items():
            if key not in df:
                df[key] = pd.NA
            df.at[index, key] = value
        
        # Print the updated DataFrame row to see the changes
        print(df.loc[index])
        print(df)
        print(index)
        
        # Optionally, save the DataFrame periodically or after each update
        df.to_csv('gpt_labels_10.csv', index=False) 

Completed call to API
created_at_fuzzy                                          2020-07-25T01:39:56Z
state_likely_tweeted_from                                              Alabama
text                         @politico I'm old enough to remember when Trum...
source                                                     Twitter for Android
display_text_width                                                          77
lang                                                                        en
favorite_count                                                               0
retweet_count                                                                0
is_quote                                                                 False
followers_count                                                             22
friends_count                                                              198
listed_count                                                                 0
statuses_count                